In [1]:
import numpy as np
import cv2
import os
from sklearn.svm import SVC
from keras_facenet import FaceNet
from mtcnn import MTCNN
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

2024-11-07 19:26:22.324455: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

# Initialize FaceNet and MTCNN
embedder = FaceNet()
detector = MTCNN()

# Step 1: Load and Generate Embeddings for Your Face Dataset
face_embeddings = []
labels = []

folder = '/Users/rushikesh/Desktop/Jay/vscode_foml/project/facedata/jay'

# Assuming 'your_face_folder' contains cropped images of your face
for filename in os.listdir(folder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(folder, filename)
        try:
            image = cv2.imread(img_path)
            image_resized = cv2.resize(image, (160, 160))
            image_normalized = image_resized.astype('float32') / 255.0
            image_resized = np.expand_dims(image_resized, axis=0)
            embedding = embedder.embeddings(image_resized)[0]
            face_embeddings.append(embedding)
            labels.append(1)  # Label '1' for your face
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    
notMyFaceFolder = '/Users/rushikesh/Desktop/Jay/vscode_foml/project/facedata/notjay_cropped'
# Optionally, add embeddings for negative samples (not your face)
for filename in os.listdir(notMyFaceFolder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(notMyFaceFolder, filename)
        try:
            image = cv2.imread(img_path)
            image_resized = cv2.resize(image, (160, 160))
            image_normalized = image_resized.astype('float32') / 255.0
            image_resized = np.expand_dims(image_resized, axis=0)
            embedding = embedder.embeddings(image_resized)[0]
            face_embeddings.append(embedding)
            labels.append(0)  # Label '0' for not your face
        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Convert to numpy arrays
face_embeddings = np.array(face_embeddings)
labels = np.array(labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━

In [3]:
# Step 2: Train an SVM Classifier
# X_train, X_test, y_train, y_test = train_test_split(face_embeddings, labels, test_size=0.2, random_state=42)

svm = SVC(kernel='linear', probability=True)
svm.fit(face_embeddings, labels)

# Evaluate SVM
# y_pred = svm.predict(X_test)
# print("SVM Accuracy:", accuracy_score(y_test, y_pred))

SVC(kernel='linear', probability=True)

In [4]:
from sklearn.svm import SVC
import joblib  # Import joblib for saving/loading the model

# Assuming `X_train` contains your embeddings and `y_train` contains the labels (1 for your face, 0 for others)
# svm = SVC(kernel='linear', probability=True)  # Set `probability=True` if you need probabilities later
# svm.fit(X_train, y_train)

# Save the SVM model
joblib.dump(svm, "svm_model.pkl")
print("Model saved successfully as 'svm_model.pkl'")


Model saved successfully as 'svm_model.pkl'


In [ ]:
# Step 3: Test on an Image with Multiple Faces
test_image_path = '/Users/rushikesh/Desktop/Jay/vscode_foml/group.jpeg'
test_image = cv2.imread(test_image_path)
test_image_rgb = cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB)

# Detect faces in the test image
faces = detector.detect_faces(test_image_rgb)

found_my_face = False

for face in faces:
    x, y, w, h = face['box']
    face_crop = test_image_rgb[y:y+h, x:x+w]  # Crop face region

    # Resize and generate embedding for each face in test image
    face_crop_resized = cv2.resize(face_crop, (160, 160))
    face_crop_resized = np.expand_dims(face_crop_resized, axis=0)
    face_embedding = embedder.embeddings(face_crop_resized)[0]

    # Use SVM to predict if the face is yours
    prediction = svm.predict([face_embedding])
    if prediction == 1:
        found_my_face = True
        cv2.rectangle(test_image, (x, y), (x+w, y+h), (0, 255, 0), 2)  # Bounding box in green
        cv2.putText(test_image, "My Face", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    else:
        cv2.rectangle(test_image, (x, y), (x+w, y+h), (0, 0, 255), 2)  # Bounding box in red

# Display result
plt.imshow(test_image)
plt.axis('off')
plt.show()


if found_my_face:
    print("Your face was found in the image!")
else:
    print("Your face was not found in the image.")

In [ ]:
input_folder = "/Users/rushikesh/Desktop/Jay/vscode_foml/project/input_folder"
output_folder = "/Users/rushikesh/Desktop/Jay/vscode_foml/project/output_folder"

os.makedirs(output_folder, exist_ok=True)

# Iterate through each image in the input folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    image = cv2.imread(image_path)
    
    if image is None:
        continue  # Skip non-image files

    # Detect faces in the image
    detections = detector.detect_faces(image)
    face_found = False

    for detection in detections:
        x, y, width, height = detection['box']
        face = image[y:y+height, x:x+width]  # Crop the face region

        # Generate embedding for the cropped face
        embedding = embedder.embeddings(np.expand_dims(face, axis=0))

        # Use the SVM model to predict if this is your face
        prediction = svm.predict(embedding)

        if prediction == 1:  # Assuming label `1` represents your face
            face_found = True
            break

    # If your face is found in the image, save it to the output folder
    if face_found:
        cv2.imwrite(os.path.join(output_folder, image_name), image)

print(f"Images with your face have been saved to '{output_folder}'")

In [ ]:
embedder = FaceNet()
detector = MTCNN()

notFaceEmbeddings = []
labels = []
notMyFaceFolder = '/Users/rushikesh/Desktop/Jay/vscode_foml/project/facedata/notjay_cropped'
# Optionally, add embeddings for negative samples (not your face)
for filename in os.listdir(notMyFaceFolder):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(notMyFaceFolder, filename)
        try:
            image = cv2.imread(img_path)
            image_resized = cv2.resize(image, (160, 160))
            image_normalized = image_resized.astype('float32') / 255.0
            image_resized = np.expand_dims(image_resized, axis=0)
            embedding = embedder.embeddings(image_resized)[0]
            notFaceEmbeddings.append(embedding)
            labels.append(0)  # Label '0' for not your face
        except Exception as e:
            print(f"Error processing {filename}: {e}")
    
# Convert to numpy arrays
notFaceEmbeddings = np.array(notFaceEmbeddings)
labels = np.array(labels)
# np.save('not_face_embeddings.npy', notFaceEmbeddings)
# print("Embeddings saved successfully as 'not_face_embeddings.npy'")

In [5]:
np.save('not_face_embeddings.npy', notFaceEmbeddings)
np.save('labels.npy', labels)